In [1]:
from symred.symplectic_form import PauliwordOp
from symred.S3_projection import S3_projection, gf2_gaus_elim, gf2_basis_for_gf2_rref, StabilizerOp

In [2]:
import numpy as np
import openfermion as of
import openfermionpyscf as ofpyscf

# comment out due to incompatible versions of Cirq and OpenFermion in Orquestra
def QubitOperator_to_dict(op, num_qubits):
    assert(type(op) == of.QubitOperator)
    op_dict = {}
    term_dict = op.terms
    terms = list(term_dict.keys())

    for t in terms:    
        letters = ['I' for i in range(num_qubits)]
        for i in t:
            letters[i[0]] = i[1]
        p_string = ''.join(letters)        
        op_dict[p_string] = term_dict[t]
         
    return op_dict

# Two examples are provided -- comment out as necessary

# One for which Hartree-Fock identifies the correct symmetry sector...
geometry=[
    ("O",(0,0,0)),
    ("H",(0.952519,0,0)),
    ("H",(-0.246530058,0.9200627021,0))
]

# ...and one for which it does not.
#geometry=[
#    ('H', (0.0,0.0,0.0)),
#    ('H', (2.45366053071732,0.0,0.0)),
#    #('H', (2.45366053071732,2.45366053071732,0.0)),
#    #('H', (0.0,2.45366053071732,0.0))
#     ]    
    
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
#molecule.load()

# Run pyscf.
molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=1,run_ccsd=1,run_fci=1)

n_qubits    = 2*molecule.n_orbitals
n_electrons = molecule.n_electrons

ham_fermionic = of.get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_jw = of.jordan_wigner(ham_fermionic)

ham_dict = QubitOperator_to_dict(ham_jw, n_qubits)

print('Jordan-Wigner Hamiltonian:\n\n', ham_dict)

Jordan-Wigner Hamiltonian:

 {'IIIIIIIIIIIIII': (-46.401782018380246+0j), 'ZIIIIIIIIIIIII': (12.413911714677475+0j), 'YZYIIIIIIIIIII': (0.12506426575287138+0j), 'XZXIIIIIIIIIII': (0.12506426575287138+0j), 'YZZZZZYIIIIIII': (-0.042709517791887974+0j), 'XZZZZZXIIIIIII': (-0.042709517791887974+0j), 'YZZZZZZZZZYIII': (0.07314533074510879+0j), 'XZZZZZZZZZXIII': (0.07314533074510879+0j), 'IZIIIIIIIIIIII': (12.413911714677475+0j), 'IYZYIIIIIIIIII': (0.12506426575287136+0j), 'IXZXIIIIIIIIII': (0.12506426575287136+0j), 'IYZZZZZYIIIIII': (-0.04270951779188799+0j), 'IXZZZZZXIIIIII': (-0.04270951779188799+0j), 'IYZZZZZZZZZYII': (0.07314533074510873+0j), 'IXZZZZZZZZZXII': (0.07314533074510873+0j), 'IIZIIIIIIIIIII': (1.6577822216057763+0j), 'IIYZZZYIIIIIII': (0.11811415718603016+0j), 'IIXZZZXIIIIIII': (0.11811415718603016+0j), 'IIYZZZZZZZYIII': (-0.2853486154987304+0j), 'IIXZZZZZZZXIII': (-0.2853486154987304+0j), 'IIIZIIIIIIIIII': (1.657782221605776+0j), 'IIIYZZZYIIIIII': (0.11811415718603013+0j), '

In [3]:
def symmetry_generators(pauliop):
    """ Find an independent basis for the Hamiltonian symmetry
    This is carried out in the symplectic representation.
    """
    # swap order of XZ blocks in symplectic matrix to ZX
    ZX_symp = np.hstack([pauliop.Z_block, pauliop.X_block])
    reduced = gf2_gaus_elim(ZX_symp)
    kernel  = gf2_basis_for_gf2_rref(reduced)

    return StabilizerOp(kernel, np.ones(kernel.shape[0]))

def measure_operator(pauli, ref_state):
    """ Measure a single Pauli operator in a single basis state
    """
    outcome=+1
    assert(len(pauli)==len(ref_state))
    for P,bit in zip(pauli, ref_state):
        if P=='Z' and bit==1:
            outcome*=-1
    return outcome

def identify_symmetry_sector(symmetry_ops,ref_state):
    """ Given the specified reference state, determine the
    correspinding sector by measuring the symmetry generators
    """
    sector = [measure_operator(pauli, ref_state) 
                for pauli in symmetry_ops]
    return sector

In [4]:
H2O_op = PauliwordOp(ham_dict)
stabs = symmetry_generators(H2O_op)
print(stabs)

(1+0j) ZIZIIZZIIZZIIZ +
(1+0j) IZIZIZIZIZIZIZ +
(1+0j) IIIIZZIIIIIIZZ +
(1+0j) IIIIIIIIZZIIII


In [5]:
proj_op = S3_projection(stabs, 'Z')
print(proj_op.rotated_stabilizers)

(-1+0j) IIIIIIIIZIIIII +
(-1+0j) IIIIZIIIIIIIII +
(-1+0j) IZIIIIIIIIIIII +
(-1+0j) ZIIIIIIIIIIIII


In [6]:
project_op = proj_op.perform_projection(H2O_op, [-1,1,-1,1])
project_op.to_sparse_matrix

<1024x1024 sparse matrix of type '<class 'numpy.complex128'>'
	with 57814 stored elements in Compressed Sparse Row format>

In [9]:
print(proj_op.stabilizers)

(1+0j) ZIZIIZZIIZZIIZ +
(1+0j) IZIZIZIZIZIZIZ +
(1+0j) IIIIZZIIIIIIZZ +
(1+0j) IIIIIIIIZZIIII


In [7]:
stab_positions = np.einsum("ij->j",proj_op.stabilizers.symp_matrix[:,:-1])
np.where(stab_positions)[0]

array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26])